## Created by yunsuxiaozi 2024/3/11
### 这里将实现sha256算法.
#### github 常用命令
(https://zhuanlan.zhihu.com/p/637271164?utm_id=0)

git init

git config user.email "helloworld@hh.com"

git config user.name "yunsuxiaozi"

git add .

git commit -m "sha256"

git branch -M main

git remote add algorithm https://github.com/yunsuxiaozi/Implementing-algorithms-from-scratch.git

git push -u algorithm main

#### 这里是循环右移的函数,比如 x=1000 0000 0000 0000 0000 0000 0000 0000 右移4位变成   0000 1000 0000 0000 0000 0000 0000 0000,即:ROTR(x,n=4)

In [1]:
#这是循环右移的函数.
inf=0xFFFFFFFF#32位无符号整数的最大值(2^32-1)
def ROTR(x, n):#Rotate Right
    x=x&inf#如果输入的数据太大,会限制到2^32-1的范围内.
    n=n%32#如果n>32,则限制到32以内的范围.
    #x=[a1,a2,……,a32]  则x>>n =[0,0,……,0,a1,a2,……,a32-n] 
    #x<<(32-n) =[a32-n+1,……a32,0,0,……,0]
    x = (x >> n) | (x << (32 - n))
    return x

#### sha256算法实现细节:

##### 1.初始哈希值H取自自然数中前面8个素数(2,3,5,7,11,13,17,19)的平方根的小数部分, 并且取前面的32位.固定常数K,取自自然数中前面64个素数的立方根的小数部分的前32位.这里可以通过注释掉的代码进行验证.

##### 2.utf-8 能够表示世界上几乎所有的字符系统,将字符串表示成二进制的数字,因为计算机只能处理数字.

##### 3.数据的padding.由于之后对数据需要按512位分块处理,所以需要将数据padding成512位的倍数.字符串本身的信息占位为$len(binaries)*8$,b'\x80' 是8位,b'\x00'\*(64\*chunks-len(binaries)-1-8) 是8\*(64\*chunks-len(binaries)-1-8)位,最后的字符串的长度信息(这里不是字符串的信息)为64位(2^64-1),计算可以发现得到的M为512的倍数.

##### 4.最后对每个chunk进行处理,先将M存储进W[0:16],进行一大堆逻辑运算的操作将字符串的信息进行混合,然后将字符串的信息和初始哈希值的信息进行交互得到混合的信息,我没有仔细思考过这个算法是为什么这样设计的,目前我感觉只是通过复杂性保证了安全性.

##### 如果还有其他细节,读者可以看代码理解.

In [2]:
def sha_256(origin_str=''):
    #初始哈希值取自自然数中前面8个素数(2,3,5,7,11,13,17,19)的平方根的小数部分, 并且取前面的32位.
    H = [0x6a09e667,0xbb67ae85,0x3c6ef372,0xa54ff53a,0x510e527f,0x9b05688c,0x1f83d9ab,0x5be0cd19]
    # prime=[2,3,5,7,11,13,17,19]
    # for idx in range(len(prime)):
    #     num=(prime[idx])**(1/2)
    #     dot=num-int(num)
    #     print(dot,H[idx]/2**32)
    
    #固定常数,取自自然数中前面64个素数的立方根的小数部分的前32位
    K = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]
    
    # prime=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
    # 31, 37, 41, 43, 47, 53, 59, 61, 67, 71,
    # 73, 79, 83, 89, 97, 101, 103, 107, 109, 113,
    # 127, 131, 137, 139, 149, 151, 157, 163, 167, 173,
    # 179, 181, 191, 193, 197, 199, 211, 223, 227, 229]
    # for idx in range(len(prime)):
    #     num=(prime[idx])**(1/3)
    #     dot=num-int(num)
    #     print(dot,K[idx]/2**32)
    
    #utf-8 能够表示世界上几乎所有的字符系统.
    binaries = origin_str.encode('utf8')
    
    """
    \x80(16进制)=128(10进制)=1000 0000(2进制,8位)
    b'\x00'=00000000(2进制 8位)
    len(binaries)是字节串的长度,即有多少字节,*8是每个字节由8位组成,转换成位长度
    to_bytes(8)转换成8字节或者说64位的字节串.byteorder='big'是大端字节序(就是从头到尾存储).

    位长度: len(binaries)*8+8位+8位*(64-len(binaries)-1-8)+64位=512.

    """
    chunks=1#M是512的多少倍

    while 64*chunks-len(binaries)-9<0:
        chunks+=1
    #M是512的整数倍
    M = binaries + b'\x80' + b'\x00'*(64*chunks-len(binaries)-1-8) + (len(binaries)<<3).to_bytes(8, byteorder='big')
    
    for chunk in range(chunks):
        M_copy=M[chunk*64:(chunk+1)*64]
        W = [0] * 64#用于存储16个32位字(每个字占用4个字节),初始化为0.
        #W[0]~W[15] 这里已经把 这个chunk的二进制转成整数
        for t in range(0, 16):
            W[t] = M_copy[t*4:(t+1)*4]#4*8=32位长度.
            #W[t]转成16进制再转成10进制.
            W[t] = int(W[t].hex(), 16)#2^32的整数.

        for t in range(16, 64):
            #^异或,位上数字相同为0,不同为1.它可以有效地混合数据，使得输出对输入的变化非常敏感
            #ROTR:这种操作有助于打乱数据，使得最终的哈希值更加均匀分布，从而提高安全性.
            # >> 这种操作进一步增加了数据的混合，有助于提高哈希函数的抗碰撞性。
            S1 = ROTR(W[t - 2], 17) ^ ROTR(W[t - 2], 19) ^ (W[t - 2]>>10)
            S0 = ROTR(W[t - 15], 7) ^ ROTR(W[t - 15], 18) ^ (W[t - 15]>>3)
            #这么多大数求和,  压缩为8位 & 0xFFFFFFFF
            W[t] = (S1+W[t-7]+S0+W[t-16]) & inf
            # 为什么要进行压缩？因为运算的时候超过8位16进制数字了
            # 压缩的目的是去掉8位前多余的数值

        a,b,c,d,e,f,g,h=H#取出初始哈希值

        for t in range(0, 64):
            S1 = ROTR(e, 6) ^ ROTR(e, 11) ^ ROTR(e, 25)
            Ch = (e & f) ^ ((~e) & g)
            S0 = ROTR(a, 2) ^ ROTR(a, 13) ^ ROTR(a, 22)
            Maj = (a & b) ^ (a & c) ^ (b & c)
            T1 = h + S1 + Ch + K[t] + W[t]
            T2 = S0 + Maj
            h = g
            g = f
            f = e
            e = (d + T1) & inf
            d = c
            c = b
            b = a
            a = (T1 + T2) & inf

        H[0] = a + H[0] & inf
        H[1] = b + H[1] & inf
        H[2] = c + H[2] & inf
        H[3] = d + H[3] & inf
        H[4] = e + H[4] & inf
        H[5] = f + H[5] & inf
        H[6] = g + H[6] & inf
        H[7] = h + H[7] & inf
    sha256 = ''
    for idx  in  range(len(H)):
        #将整数转换成4个字节(4*8=32位)的二进制数据,然后再转成十六进制数据(hex)
        sha256 = sha256 + H[idx].to_bytes(4, byteorder='big').hex()
    return sha256

##### 算法的检验:算法的检验比较简单粗暴,就是随机生成一大堆数据,用手写的算法和标准库的算法进行比较,如果准确率为1的话就没有问题了.目前在小范围测试是没有问题的.

In [3]:
import random#用于生成随机数的库
import hashlib#使用哈希算法对数据加密的库
seed=2024
random.seed(seed)
num_string=10000
char_num="abcdefghijklmnopqrstuvwxyz0123456789"#可供选择的字符是a~z加上数字
random_strings = []#存储随机生成的字符串

for _ in range(num_string):
    str_len = random.randint(10, 500)  #生成长度为10到500的字符
    random_string = ''.join(random.choices(char_num, k=str_len))
    random_strings.append(random_string)
true_cnt=0#手写算法正确的个数
for idx in range(num_string):
    origin_str=random_strings[idx]
    #hashlib.sha256(binaries).hexdigest() 对二进制数据binaries经过sha-256算法计算后的十六进制哈希值
    if sha_256(origin_str) == hashlib.sha256(origin_str.encode('utf8')).hexdigest():
        true_cnt+=1
print(f"accuracy:{true_cnt/num_string}")

accuracy:1.0
